integrated_test_00
----

Check ray generation for plasma sources. In particular we want to check that emissivity to ray calculations are being performed correctly including solid angle and bundle volume calculations.

In [ ]:
%matplotlib notebook
import numpy as np
import xicsrt

In [ ]:
# 1.
config = dict()

# 2.
config['general'] = {}
config['general']['number_of_iter'] = 1
config['general']['number_of_runs'] = 1

# 3.
config['sources'] = {}
config['sources']['source'] = {}
config['sources']['source']['class_name'] = 'XicsrtPlasmaCubic'
config['sources']['source']['origin'] = [0.0, 0.0, 0.0]
config['sources']['source']['xsize'] = 0.01
config['sources']['source']['ysize'] = 0.01
config['sources']['source']['zsize'] = 0.01
config['sources']['source']['target'] = [0.0, 0.0, 1.0]

config['sources']['source']['emissivity'] = 1e12
config['sources']['source']['time_resolution'] = 1
config['sources']['source']['spread'] = np.radians(180)


# 4.
config['optics'] = {}
config['optics']['detector'] = {}
config['optics']['detector']['class_name'] = 'XicsrtOpticDetector'
config['optics']['detector']['origin'] = [0.0,  0.0, 1.0]
config['optics']['detector']['zaxis']  = [0.0, 0.0, -1.0]
config['optics']['detector']['xsize']  = 0.1
config['optics']['detector']['ysize']  = 0.1
config['optics']['detector']['check_size'] = False

# 6.
results = xicsrt.raytrace(config)

In [ ]:
num_expected = (config['sources']['source']['emissivity']
                * config['sources']['source']['xsize']
                * config['sources']['source']['ysize']
                * config['sources']['source']['zsize'])
num_actual = results['total']['meta']['source']['num_out']
print(f'Rays generated at source')
print(f'  num_expected: {num_expected} num_actual: {num_actual}')
np.testing.assert_allclose(num_expected, num_actual, 1)

num_exp_detector = num_expected/2
num_act_detector = results['total']['meta']['detector']['num_out']
print(f'Rays on detector plane')
print(f'  num_expected: {num_exp_detector} num_actual: {num_act_detector}')
sigma_5 = np.sqrt(num_exp_detector)*5
np.testing.assert_allclose(num_exp_detector, num_act_detector, sigma_5)